<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/Write_Codes_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Read into one dataset and get its indexes

Note: get the exact file name to read

In [ ]:
import chardet
with open('DOMComprasCardio.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.99, 'encoding': 'utf-8', 'language': ''}

In [ ]:
import pandas as pd
OrgData = pd.read_csv('DOMComprasCardio.csv',encoding='utf-8')
OrgData.head(1)

,Unnamed: 0,CODIGO_UNIDAD_COMPRA,UNIDAD_COMPRA,CODIGO_PROCESO,MODALIDAD,ESTADO_PROCESO,FECHA_PUBLICACION,ID_ARTICULO,SUBCLASE_UNSPSC,DESCRIPCION_SUBCLASE,DESCRIPCION_ARTICULO,CANTIDAD,ABR_UNIDAD_MEDIDA,DESCRIPCION_UNIDAD_MEDIDA,PRECIO_UNITARIO_ESTIMADO,MONTO_TOTAL_ESTIMADO,CUENTA_PRESUPUESTARIA
0,1,195,Sistema Unico de Beneficiarios (SIUBEN),SIUBEN-DAF-CM-2019-0013,Compras Menores,Proceso adjudicado y celebrado,2019-03-13 15:00:06,12462773,51121709,Carvedilol,"Carvedilol 25 MG, 30/1",5.0,CAJ,Caja,450,2250.0,2.3.4.1.01


Note: get the special column name to get the dirty names

In [ ]:
OrgDataDirtyName = OrgData['DESCRIPCION_ARTICULO']

In [ ]:
len(OrgDataDirtyName) # length of drug names

3077

In [ ]:
OrgDataDirtyName = list(set(OrgDataDirtyName))
len(OrgDataDirtyName) # length of drug names after remove duplicates

1959

In [ ]:
# uppercase everything
OrgDataDirtyNameU = []
for i in OrgDataDirtyName:
  OrgDataDirtyNameU.append(i.upper())

In [ ]:
OrgDataDirtyNameU[:5]

['CANDESARTAN CILEXETILO 32 MGX 30 TAB.',
 'SIMVASTATINA 20MG 100 TABS',
 'LOSARTAN H 100/25MG TAN ETHICAL',
 'LOSARTAN H 100/25',
 '109-10639-CAPTOPRILO; 25 MG; COMPRIMIDO ORAL SUBLINGUAL; BLISTER']

translate from Spainish to English

In [ ]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = OrgDataDirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 1.4 MB 10.7 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=1a020d2e41a9b14d84a668258f3022c491c64ea0615a3c8316ee661f30f20941
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
CANDESARTAN CILEXETILO 32 MGX 30 TAB.


In [ ]:
OrgDataDirtyNameUEng = []
for item in OrgDataDirtyNameU:
  translation = translator.translate(item, dest='en')
  OrgDataDirtyNameUEng.append(translation.text)
# runs 1 mins

In [ ]:
OrgDataDirtyNameUEng[:5]

['CANDESARTAN CILEXETILO 32 MGX 30 TAB.',
 'SIMVASTATINA 20MG 100 TABS',
 'LOSARTAN H 100/25MG TAN ETHICAL',
 'LOSARTAN H 100/25',
 '109-10639-CAPTOPRILO; 25 MG; COMPRIMIDO ORAL SUBLINGUAL; BLISTER']

In [ ]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

OrgDataDirtyNameUESpl = []
for item in OrgDataDirtyNameUEng:
  tokenlist = tokenizer.tokenize(item)
  OrgDataDirtyNameUESpl.append(tokenlist)

In [ ]:
OrgDataDirtyNameUESpl[:1]

[['CANDESARTAN', 'CILEXETILO', '32', 'MGX', '30', 'TAB', '.']]

In [ ]:
OrgDataDirtyNameIndex  = []
for item in OrgDataDirtyNameUESpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  OrgDataDirtyNameIndex.append(templist)

In [ ]:
OrgDataDirtyNameIndex[:5]

[['CANDESARTAN', 'CILEXETILO'],
 ['SIMVASTATINA'],
 ['LOSARTAN', 'ETHICAL'],
 ['LOSARTAN'],
 ['CAPTOPRILO', 'COMPRIMIDO', 'SUBLINGUAL', 'BLISTER']]

In [ ]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
OrgDataDirtyNameIndexDel = []
for item in OrgDataDirtyNameIndex:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  OrgDataDirtyNameIndexDel.append(add)

In [ ]:
OrgDataDirtyNameIndexDel[:5]

[['CANDESARTAN', 'CILEXETILO'],
 ['SIMVASTATINA'],
 ['LOSARTAN', 'ETHICAL'],
 ['LOSARTAN'],
 ['CAPTOPRILO', 'COMPRIMIDO', 'SUBLINGUAL', 'BLISTER']]

## Part 2: Set up indexes for clean catagol drug names

In [ ]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [ ]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [ ]:
CatalogNameUS[:5] # after split by ","

[['METOTREXATE', ' INJECTION. 500MG/20ML ROAD BOTTLE'],
 ['INSULIN HUMAN 100 UI/ML', ' SOLUTION FOR INJECTION', ' TBD'],
 ['DARUNAVIR 800 MG', ' TABLET', ' 60 TABLETS BOTTLE'],
 ['GLIMEPIRIDE 2 MG', ' TABLET', ' TBD'],
 ['INSULIN GLULISINE 100 UI/ML', ' SOLUTION FOR INJECTION', ' TBD']]

In [ ]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [ ]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [ ]:
PresentationForm[:5]# the presentation form

[' CAPSULE',
 ' TABLET (DISPERSIBLE',
 ' POWDER FOR INJECTION. 500MCG ROAD BOTTLE',
 '\xa0TABLET',
 ' SOLUTION FOR INJECTABLE. 20MG/10ML BOTTLE']

In [ ]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [ ]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [ ]:
PresentationDiction2 = list(set(PresentationDiction2))

In [ ]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [ ]:
print(len(MainMedicine), MainMedicine[:5])

462 ['METOTREXATE', 'INSULIN HUMAN 100 UI/ML', 'DARUNAVIR 800 MG', 'GLIMEPIRIDE 2 MG', 'INSULIN GLULISINE 100 UI/ML']


In [ ]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [ ]:
MainMedicineSpl[:5]

[['METOTREXATE'],
 ['INSULIN', 'HUMAN', '100', 'UI', '/', 'ML'],
 ['DARUNAVIR', '800', 'MG'],
 ['GLIMEPIRIDE', '2', 'MG'],
 ['INSULIN', 'GLULISINE', '100', 'UI', '/', 'ML']]

In [ ]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)

In [ ]:
MainMedicineIndex[:5]

[['METOTREXATE'],
 ['INSULIN', 'HUMAN'],
 ['DARUNAVIR'],
 ['GLIMEPIRIDE'],
 ['INSULIN', 'GLULISINE']]

In [ ]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [ ]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [ ]:
len(CatalogIndexData_df)

462

In [ ]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"METOTREXATE, INJECTION. 500MG/20ML ROAD BOTTLE",[METOTREXATE]
1,"INSULIN HUMAN 100 UI/ML, SOLUTION FOR INJECTIO...","[INSULIN, HUMAN]"
2,"DARUNAVIR 800 MG, TABLET, 60 TABLETS BOTTLE",[DARUNAVIR]
3,"GLIMEPIRIDE 2 MG, TABLET, TBD",[GLIMEPIRIDE]
4,"INSULIN GLULISINE 100 UI/ML, SOLUTION FOR INJE...","[INSULIN, GLULISINE]"


In [ ]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [ ]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [ ]:
CatalogMedName[:3]

['ACICLOVIR', 'AMPHOTERICIN', 'RALTEGRAVIR']

## Part 3: Write the Index Data Set According to the Catalog Name Index

In [ ]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 2 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [ ]:
DatafinIndex = []
for sublist in OrgDataDirtyNameIndexDel:
  add = KeepMetAnd(sublist)
  DatafinIndex.append(add)

In [ ]:
# count the number of Chile index sub list which is null []
count = 0
for i in DatafinIndex:
  if not i:
    count += 1

the length of total drug names and null index

In [ ]:
print(len(DatafinIndex), count) 

1959 464


In [ ]:
nullindexlist = []
for i in range(0, len(DatafinIndex)):
  if not DatafinIndex[i]:
    nullindexlist.append(i)

In [ ]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.9 MB/s 
     |████████████████████████████████| 71 kB 4.3 MB/s 
     |████████████████████████████████| 6.0 MB 40.7 MB/s 
     |████████████████████████████████| 13.5 MB 20.5 MB/s 
     |████████████████████████████████| 188 kB 46.9 MB/s 
     |████████████████████████████████| 660 kB 20.1 MB/s 
     |████████████████████████████████| 10.1 MB 34.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found ex

In [ ]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.3 MB/s 
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=e36ab9514a28eaaf2555613dbdb018116440b4b695618a07e69ae78527de1533
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scisp

In [ ]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [ ]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

use spacy nlp to get informations from the drug names which with null index

In [ ]:
DatadrugOutDataName = []
for nullindex in nullindexlist:
  tempdrugname = OrgDataDirtyNameIndexDel[nullindex]
  tempdrugname_str = str(' '.join(tempdrugname))
  doc = nlp(tempdrugname_str)
  temptext = doc.text
  DatadrugOutDataName.append(temptext)

In [ ]:
drugOutDataNameNullNumber = 0

for i in DatadrugOutDataName:
  if not i:
    drugOutDataNameNullNumber += 1

the number of drug name which are still null

In [ ]:
drugOutDataNameNullNumber

0

In [ ]:
# for these null element in list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  DatafinIndex[i] = DatadrugOutDataName[j]
  j += 1

In [ ]:
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    DatafinIndex[index] = OrgDataDirtyNameIndexDel[index]

In [ ]:
nullindex = []
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    nullindex.append(index)

In [ ]:
nullindex

[]

In [ ]:
PreList = []

for item in OrgDataDirtyNameIndexDel:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  PreList.append(list(set(addlist)))

In [ ]:
WeightList = []

for item in OrgDataDirtyNameIndexDel:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [ ]:
print(len(DatafinIndex), len(PreList), len(WeightList))

1959 1959 1959


In [ ]:
# for catalog name index dataframe, transform list to string
str_DatafinIndex = []
for item in DatafinIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_DatafinIndex.append(newitem)

In [ ]:
str_DatafinIndex[:4]

['CANDESARTAN', 'SIMVASTATINA', 'LOSARTAN', 'LOSARTAN']

In [ ]:
str_PreList = []

for item in PreList:
  newitem = ' '.join(item)
  str_PreList.append(newitem)

In [ ]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [ ]:
DrugDataWrite = {"DOMComprasCardio Dirty Name English": OrgDataDirtyNameUEng, "Final Drug Index": str_DatafinIndex, "Presentation Form": str_PreList, "Weight": str_WeightList}


In [ ]:
DrugDataWrite_df = pd.DataFrame(DrugDataWrite)

In [ ]:
DrugDataWrite_df

,DOMComprasCardio Dirty Name English,Final Drug Index,Presentation Form,Weight
0,CANDESARTAN CILEXETILO 32 MGX 30 TAB.,CANDESARTAN,,
1,SIMVASTATINA 20MG 100 TABS,SIMVASTATINA,,
2,LOSARTAN H 100/25MG TAN ETHICAL,LOSARTAN,,
3,LOSARTAN H 100/25,LOSARTAN,,
4,109-10639-CAPTOPRILO; 25 MG; COMPRIMIDO ORAL S...,CAPTOPRILO COMPRIMIDO SUBLINGUAL BLISTER,,
...,...,...,...,...
1954,ATENOLOL 50 MG. TAB.,ATENOLOL,,
1955,ATENOLOL 50MG X 90 COMPS,ATENOLOL,,
1956,BISOPROLOL 5MG TABS.,BISOPROLOL,,
1957,LOSARTAN TABLET,LOSARTAN,TABLET,


In [ ]:
DrugDataWrite_df.to_excel("DOMComprasCardio (Write).xlsx", sheet_name = 'DOMComprasCardio')  